In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\open_ai_assistant_v2")
from src.components.chains.chains import Assistant
import pandas as pd

assistant = Assistant(conversation_id="8af20de4-cd11-41da-9a9d-04762dd91322")

#### Generando un resumen de la conversación

In [2]:
assistant.add_user_message(message="Dame el promedio de la presion estatica del computador de flujo con tag FQI-EMED-3135.01-067 para los meses de agosto del 2023")
# assistant.add_ai_message(message="Aqui tienes la lista de los sistemas de Medicion: Medidor 1a, Medidor 2a, Medidor 3a")
# assistant.add_user_message(message="Hola")
# assistant.add_ai_message(message="Claro, ¿me puedes proporcionar un identificador del sistema de medición?")
# assistant.add_user_message(message="me dices cuales son los ids que existen?")
# assistant.add_user_message(message="el Tag es F980-40")
# assistant.add_ai_message(message="Por supuesto, ¿que tipo de información requieres?")
# assistant.add_user_message(message="Me gustaria saber el promedio de la presion en las fechas de enero y febrero")
# assistant.add_user_message(message="Necesito una lista de los sistemas de medicion de gas")
summary = assistant.get_memory_summary()
print(summary)


        The conversation is about obtaining the average static pressure of the flow computer with tag FQI-EMED-3135.01-067 for the months of August 2023.


#### Generamos el requerimiento

In [3]:
user_request = assistant.get_request(summary)
print(user_request["request"])

The human is asking for the average static pressure of the flow computer with Tag FQI-EMED-3135.01-067 for the months of August 2023.


#### Obtenemos las keywords

In [4]:
kw = assistant.get_keywords_from_requirement(user_request["request"])
kw

['variables', 'pressure', 'flow computer', 'average', 'month', '2023']

#### Datos de tablas

In [5]:
r = assistant.get_related_data_tables(['variables', 'static pressure', 'flow computer', 'tag'])
print(r)


[('var_variable_datos', "The table var_variable_datos contains read/measured/calculated values/information for variables in table var_tipo_variable. They are classified by the measurement system through the idSistemaMedicion_fk column and the variable through the idVariable_fk column.'", 'CREATE TABLE IF NOT EXISTS dbo_v2.var_variable_datos (\n    Fecha DATETIME,\n    idVariable_fk INT,\n    idSistemaMedicion_fk INT,\n    Valor FLOAT,\n    Valor_String VARCHAR(100),\n    PRIMARY KEY (Fecha, idVariable_fk, idSistemaMedicion_fk),\n    FOREIGN KEY (idVariable_fk) REFERENCES dbo_v2.var_tipo_variable (Id),\n    FOREIGN KEY (idSistemaMedicion_fk) REFERENCES dbo_v2.med_sistema_medicion (Id)\n)'), ('fcs_computador_medidor', 'The fcs_computador_medidor table connect fcs_computadores table and med_sistema_medicion with IdComputador_fk that is a foreign key referencing the Id column in the fcs_computadores table and the Id_Sisema_Medicion is a foreign key referencing the Id column in the med_sist

#### Clasificamos el requerimiento simple-complejo

In [6]:
request_type = assistant.simple_filter(user_request["request"])
print(request_type)

{'analysis': 'This input is out of my knowledge, this type is complex', 'response': 'complex'}


#### Clasificamos el requerimiento complejo (completo-incompleto)

In [1]:
request_type = assistant.complex_filter(user_request["request"], r)
print(request_type)

NameError: name 'assistant' is not defined

In [12]:
print(request_type["text"])


        analysis: The request is complete as it specifies the flow computer with Tag FQI-EMED-3135.01-067 and the time period of August 2023.
        type: complete
